In [1]:
from keras.models import Model
from keras.layers import Input, Lambda, Dense, Flatten
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tensorflow


In [3]:
print(tensorflow.__version__)

2.5.0


In [4]:
IMAGE_SIZE = [224, 224]

In [5]:
train_path = './Images/Train/'
test_path = './Images/Test/'

In [6]:
vgg = VGG16(input_shape = IMAGE_SIZE + [3],  weights='imagenet', include_top = False)

In [7]:
for layer in vgg.layers:
    layer.trainable = False

In [8]:
categories = glob('./Images/Train/*')

In [9]:
print(categories)

['./Images/Train\\Rajasekhar', './Images/Train\\srinithin']


In [10]:
flatten_layer = Flatten()(vgg.output)

In [11]:
last_layer = Dense(len(categories), activation = 'softmax')(flatten_layer)

In [12]:
model = Model(inputs=vgg.input, outputs=last_layer)

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [14]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [15]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

In [16]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [17]:
training_data = train_datagen.flow_from_directory('./Images/Train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 184 images belonging to 2 classes.


In [18]:
test_data = train_datagen.flow_from_directory('Images/Test',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 157 images belonging to 2 classes.


In [19]:
print(training_data)

In [20]:
model_fit = model.fit(training_data,
                               validation_data = test_data,
                               epochs = 5,
                               steps_per_epoch = len(training_data),
                               validation_steps = len(test_data))

Epoch 1/5
6/6 [==============================] - 55s 8s/step - loss: 1.1795 - accuracy: 0.5674 - val_loss: 2.6326 - val_accuracy: 0.3631
Epoch 2/5
6/6 [==============================] - 45s 8s/step - loss: 0.4270 - accuracy: 0.8030 - val_loss: 0.1571 - val_accuracy: 0.9490
Epoch 3/5
6/6 [==============================] - 44s 8s/step - loss: 0.0204 - accuracy: 1.0000 - val_loss: 0.0653 - val_accuracy: 0.9809
Epoch 4/5
6/6 [==============================] - 45s 8s/step - loss: 0.0292 - accuracy: 1.0000 - val_loss: 0.1448 - val_accuracy: 0.9618
Epoch 5/5
6/6 [==============================] - 44s 8s/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.1097 - val_accuracy: 0.9618


In [21]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

NameError: name 'r' is not defined

In [22]:
plt.plot(model_fit.history['acc'], label='train acc')
plt.plot(model_fit.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

KeyError: 'acc'

In [23]:
import tensorflow as tf

from keras.models import load_model

model.save('facerecog_model.h5')